# Finite-element mode solver

You can mesh any component cross-section and solve the PDEs thanks to [femwell](https://helgegehring.github.io/femwell) mode solver.

Unlike other mode solvers, this actually uses the component geometry instead of a hardcoded geometry.

You can directly compute the modes of a Gdsfactory cross-section using `gplugins.femwell.mode_solver.compute_cross_section_modes`.

You can also use femwell directly with shapely polygons for more control over the geometry.

In [ ]:
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
from femwell.maxwell.waveguide import compute_modes
from femwell.mesh import mesh_from_OrderedDict
from shapely.geometry import Polygon
from shapely.ops import clip_by_rect
from skfem import Basis, ElementTriP0
from skfem.io.meshio import from_meshio

## Define waveguide geometry

First we define the waveguide cross-section geometry using shapely polygons:

In [ ]:
# Define waveguide parameters
wg_width = 0.5  # um
wg_thickness = 0.22  # um
box_thickness = 2.0  # um (buried oxide)
clad_thickness = 2.0  # um (top cladding)

# Create the waveguide core polygon
core = Polygon([
    (-wg_width / 2, 0),
    (wg_width / 2, 0),
    (wg_width / 2, wg_thickness),
    (-wg_width / 2, wg_thickness),
])

# Create surrounding regions (box and cladding)
# Use a buffer around the core for the simulation domain
buffer_size = 3.0  # um padding around waveguide

polygons = OrderedDict(
    core=core,
    box=clip_by_rect(core.buffer(buffer_size), -np.inf, -box_thickness, np.inf, 0),
    clad=clip_by_rect(core.buffer(buffer_size), -np.inf, 0, np.inf, clad_thickness),
)

print(f"Waveguide: {wg_width} x {wg_thickness} um")

## Generate mesh

We generate a mesh using femwell's mesh_from_OrderedDict function with refinement near the core:

In [ ]:
# Define mesh resolution: finer mesh in and around the core
resolutions = {
    "core": {"resolution": 0.03, "distance": 1.0},
}

# Generate the mesh
mesh = from_meshio(
    mesh_from_OrderedDict(
        polygons, resolutions, default_resolution_max=0.3
    )
)

mesh.draw().show()

## Assign material refractive indices

In [ ]:
# Define refractive indices
n_si = 3.48  # Silicon at 1.55 um
n_sio2 = 1.44  # SiO2

# Create basis and assign permittivity (n^2) to each region
basis0 = Basis(mesh, ElementTriP0())
epsilon = basis0.zeros()

for subdomain, n in {"core": n_si, "box": n_sio2, "clad": n_sio2}.items():
    epsilon[basis0.get_dofs(elements=subdomain)] = n**2

basis0.plot(epsilon, colorbar=True).show()

## Solve for modes

In [ ]:
# Solve for the waveguide modes
wavelength = 1.55  # um
num_modes = 4

modes = compute_modes(basis0, epsilon, wavelength=wavelength, num_modes=num_modes, order=1)

# Print effective indices and polarization
for i, mode in enumerate(modes):
    print(f"Mode {i}: n_eff = {mode.n_eff:.4f}, TE fraction = {mode.te_fraction:.2%}")

## Visualize mode fields

You can use femwell's built-in visualization methods to inspect the modes:

In [ ]:
# Show the fundamental mode (TE-like)
print(f"Fundamental mode TE fraction: {modes[0].te_fraction:.2%}")
modes[0].show("E", part="real")

In [ ]:
# Plot Ex component of the fundamental mode
modes[0].plot_component("E", component="x", part="real", colorbar=True)

In [ ]:
# Show second mode (TM-like if it exists)
if len(modes) > 1:
    print(f"Second mode TE fraction: {modes[1].te_fraction:.2%}")
    modes[1].show("E", part="real")

## Mode object methods

You can inspect what methods are available on the mode object:

In [ ]:
# List available attributes and methods
print([attr for attr in dir(modes[0]) if not attr.startswith("_")])

## Effective index vs width sweep

Let's sweep the waveguide width to see how the effective index changes:

In [ ]:
widths = np.linspace(0.3, 1.0, 8)
num_modes_sweep = 2
all_neffs = np.zeros((len(widths), num_modes_sweep))
all_te_fracs = np.zeros((len(widths), num_modes_sweep))

for i, w in enumerate(widths):
    # Create geometry for this width
    core_w = Polygon([
        (-w / 2, 0), (w / 2, 0),
        (w / 2, wg_thickness), (-w / 2, wg_thickness),
    ])
    polys = OrderedDict(
        core=core_w,
        box=clip_by_rect(core_w.buffer(buffer_size), -np.inf, -box_thickness, np.inf, 0),
        clad=clip_by_rect(core_w.buffer(buffer_size), -np.inf, 0, np.inf, clad_thickness),
    )
    
    # Generate mesh and compute modes
    m = from_meshio(mesh_from_OrderedDict(polys, resolutions, default_resolution_max=0.3))
    b0 = Basis(m, ElementTriP0())
    eps = b0.zeros()
    for subdomain, n in {"core": n_si, "box": n_sio2, "clad": n_sio2}.items():
        eps[b0.get_dofs(elements=subdomain)] = n**2
    
    modes_w = compute_modes(b0, eps, wavelength=wavelength, num_modes=num_modes_sweep, order=1)
    all_neffs[i] = np.real([mode.n_eff for mode in modes_w])
    all_te_fracs[i] = [mode.te_fraction for mode in modes_w]
    print(f"Width {w:.2f} um: n_eff = {all_neffs[i, 0]:.4f}")

In [ ]:
# Plot effective index vs width
plt.figure(figsize=(8, 5))
plt.xlabel("Waveguide Width (µm)")
plt.ylabel("Effective Refractive Index")

for mode_idx in range(num_modes_sweep):
    sc = plt.scatter(widths, all_neffs[:, mode_idx], c=all_te_fracs[:, mode_idx], 
                     cmap="coolwarm", vmin=0, vmax=1, s=50)
    plt.plot(widths, all_neffs[:, mode_idx], '--', alpha=0.5, label=f"Mode {mode_idx}")

plt.axhline(y=n_sio2, color='gray', linestyle=':', label='n_clad')
plt.colorbar(sc, label="TE Fraction")
plt.legend()
plt.title(f"Mode neff vs Width (λ = {wavelength} µm, h = {wg_thickness} µm)")
plt.tight_layout()
plt.show()